# Reading preprocessed data

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

fhvhv_covid_df = pd.read_csv('/Volumes/E/data/preprocessed_data/fhvhv_covid.csv')
fhvhv_covid_df

,pickup_date,num_of_SR,trip_count,avg_trip_duration,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,...,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG
0,2020-09-01,34,344879,17.833136,294,34,255,278,52,9,...,45,48,73,4,58,62,24,1,18,19
1,2020-09-02,42,357969,17.718040,271,25,249,272,42,6,...,45,48,69,3,58,62,17,0,18,19
2,2020-09-03,26,391021,18.258425,274,29,250,274,41,5,...,43,46,90,11,62,67,17,4,19,20
3,2020-09-04,6,425570,19.115868,230,19,243,267,44,1,...,40,43,59,4,63,68,26,2,20,21
4,2020-09-05,0,434656,17.152276,156,11,247,269,30,3,...,40,43,34,3,63,68,24,0,22,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2020-12-27,0,285895,15.374107,2886,920,3239,4094,479,170,...,428,540,868,317,970,1238,312,79,319,395
118,2020-12-28,6,301203,16.477568,5385,1196,3389,4266,1019,212,...,438,553,1530,390,1005,1284,562,89,338,415
119,2020-12-29,2,330461,16.430484,5201,1003,3547,4432,991,196,...,448,562,1602,318,1062,1347,584,80,369,447
120,2020-12-30,2,358647,16.910249,4975,1052,3708,4610,966,182,...,467,588,1492,328,1105,1400,465,92,379,457


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error,mean_absolute_error 
from sklearn.metrics import r2_score

from numpy import absolute
from numpy import mean
from numpy import std

from statsmodels.formula.api import ols,glm
from statsmodels.api import families

In [3]:
# train set: data from Sep to Nov
train_df = fhvhv_covid_df.loc[(fhvhv_covid_df['pickup_date'] >= '2020-09-01') & 
                          (fhvhv_covid_df['pickup_date'] <= '2020-11-30')]
train_df.describe()

,num_of_SR,trip_count,avg_trip_duration,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_CASE_COUNT_7DAY_AVG,...,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG
count,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,...,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000
mean,38.967033,405788.626374,17.512959,786.175824,216.329670,725.890110,921.054945,128.296703,36.538462,118.527473,...,118.505495,153.000000,204.010989,57.725275,187.384615,239.252747,80.901099,22.670330,72.406593,92.868132
std,31.430152,59864.531517,2.228100,598.822912,208.797193,496.798941,681.356920,109.708249,37.887643,92.414806,...,89.593325,124.668806,161.467334,57.942705,134.743606,185.708948,81.124329,24.289026,67.385290,87.486914
min,0.000000,314469.000000,15.383007,156.000000,11.000000,236.000000,259.000000,18.000000,1.000000,37.000000,...,37.000000,41.000000,28.000000,3.000000,58.000000,62.000000,13.000000,0.000000,18.000000,19.000000
25%,0.000000,357824.000000,16.865501,363.500000,46.500000,352.000000,396.500000,52.500000,8.000000,52.500000,...,51.500000,56.500000,95.000000,11.000000,80.000000,88.500000,26.000000,2.000000,24.000000,26.500000
50%,46.000000,388982.000000,17.409551,584.000000,128.000000,526.000000,611.000000,82.000000,20.000000,70.000000,...,75.000000,93.000000,140.000000,30.000000,132.000000,151.000000,38.000000,11.000000,35.000000,42.000000
75%,66.000000,437607.000000,17.900605,989.000000,379.000000,947.500000,1302.000000,171.000000,65.500000,157.500000,...,162.500000,228.000000,263.500000,101.500000,242.000000,334.000000,111.000000,44.500000,104.000000,151.000000
max,96.000000,596817.000000,36.772537,2802.000000,788.000000,1980.000000,2605.000000,456.000000,145.000000,338.000000,...,316.000000,425.000000,717.000000,227.000000,531.000000,707.000000,377.000000,93.000000,253.000000,313.000000


In [4]:
# test_set: Dec data 
test_df = fhvhv_covid_df.loc[(fhvhv_covid_df['pickup_date'] >= '2020-12-01') & 
                          (fhvhv_covid_df['pickup_date'] <= '2020-12-31')]
test_df.describe()

,num_of_SR,trip_count,avg_trip_duration,CASE_COUNT,PROBABLE_CASE_COUNT,CASE_COUNT_7DAY_AVG,ALL_CASE_COUNT_7DAY_AVG,BX_CASE_COUNT,BX_PROBABLE_CASE_COUNT,BX_CASE_COUNT_7DAY_AVG,...,MN_CASE_COUNT_7DAY_AVG,MN_ALL_CASE_COUNT_7DAY_AVG,QN_CASE_COUNT,QN_PROBABLE_CASE_COUNT,QN_CASE_COUNT_7DAY_AVG,QN_ALL_CASE_COUNT_7DAY_AVG,SI_CASE_COUNT,SI_PROBABLE_CASE_COUNT,SI_CASE_COUNT_7DAY_AVG,SI_ALL_CASE_COUNT_7DAY_AVG
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,...,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,32.903226,375152.838710,16.816636,3140.419355,863.645161,2919.709677,3753.903226,541.387097,152.612903,494.290323,...,412.322581,542.032258,916.516129,268.096774,844.516129,1099.838710,325.419355,75.064516,307.580645,381.290323
std,34.862927,50598.481111,1.097466,980.150457,152.655068,379.208491,419.820704,204.580168,32.369407,68.854045,...,35.762538,34.538369,295.612795,50.855583,132.620730,154.141731,101.801367,15.635292,32.451784,36.736171
min,0.000000,285895.000000,15.064895,1046.000000,584.000000,2132.000000,2812.000000,154.000000,84.000000,364.000000,...,328.000000,452.000000,299.000000,185.000000,575.000000,764.000000,96.000000,41.000000,263.000000,327.000000
25%,0.000000,344579.500000,16.058386,2669.000000,764.500000,2682.500000,3515.000000,425.000000,131.500000,461.500000,...,382.000000,516.500000,761.000000,228.500000,763.000000,1014.000000,254.000000,64.000000,283.500000,354.500000
50%,10.000000,368841.000000,16.652988,3061.000000,902.000000,2874.000000,3731.000000,528.000000,155.000000,474.000000,...,422.000000,553.000000,874.000000,269.000000,831.000000,1092.000000,328.000000,74.000000,300.000000,369.000000
75%,72.000000,411164.500000,17.442335,3515.000000,956.000000,3114.500000,3939.500000,626.500000,176.000000,508.500000,...,439.000000,568.000000,1040.500000,307.500000,923.500000,1175.500000,376.000000,88.000000,322.500000,402.000000
max,86.000000,484294.000000,19.624485,5385.000000,1196.000000,3761.000000,4682.000000,1019.000000,212.000000,687.000000,...,467.000000,588.000000,1602.000000,390.000000,1125.000000,1424.000000,584.000000,107.000000,400.000000,481.000000


# Fitting ridge regression model

In [8]:
X = train_df.drop(['pickup_date','trip_count'],axis=1).values
y = train_df['trip_count'].values
y = y.reshape(-1, 1)

# Standardize data
scaler = StandardScaler()
scaler.fit_transform(X)
X = scaler.transform(X)
scaler.fit(y)
y = scaler.transform(y)


# Split train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

rr = Ridge(alpha=10)
rr.fit(X_train, y_train)
pred_train = rr.predict(X_train)

# Compute MAE, MSE, R2 to evalute the performance of the model
print("MAE on train data: ", mean_absolute_error(y_train, pred_train))
print("MSE on train data: ", mean_squared_error(y_train, pred_train))
print("R2 score on train data: ", r2_score(y_train, pred_train))

pred_test = rr.predict(X_test)
print("\nMAE on test data: ", mean_absolute_error(y_test, pred_test))
print("MSE on test data: ", mean_squared_error(y_test, pred_test))
print("R2 score on test data: ", r2_score(y_test, pred_test))

MAE on train data:  0.5628173975057619
MSE on train data:  0.5315238667272311
R2 score on train data:  0.3401368323413876

MAE on test data:  0.7523612226044124
MSE on test data:  0.9701738699807506
R2 score on test data:  0.22519390166849385


# Hyperparameter tuning

In [9]:
# Grid search

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

# Define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# Define grid
grid = {'alpha':[0.01, 0.1, 1, 10]}

search = GridSearchCV(rr, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

results = search.fit(X, y)

print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

MAE: -0.706
Config: {'alpha': 10}


# Predicting future data

In [10]:
X = test_df.drop(['pickup_date','trip_count'],axis=1).values
y = test_df['trip_count'].values
y = y.reshape(-1, 1)

# Standardize data
scaler.fit(X)
X = scaler.transform(X)
scaler.fit(y)
y = scaler.transform(y)

pred_y = rr.predict(X)

# Compute MAE, MSE, R2 to evalute the performance of the model
print("MAE on future data: ", mean_absolute_error(y, pred_y))
print("MSE on future data: ", mean_squared_error(y, pred_y))
print("R2 score on future data: ", r2_score(y, pred_y))

MAE on future data:  0.6671975968424869
MSE on future data:  0.8838572402060677
R2 score on future data:  0.1161427597939323
